## Lab 2

### Part 3. Poetry generation

Let's try to generate some poetry using RNNs. 

You have several choices here: 

* The Shakespeare sonnets, file `sonnets.txt` available in the notebook directory.

* Роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина. В предобработанном виде доступен по [ссылке](https://github.com/attatrol/data_sources/blob/master/onegin.txt).

* Some other text source, if it will be approved by the course staff.

Text generation can be designed in several steps:
    
1. Data loading.
2. Dictionary generation.
3. Data preprocessing.
4. Model (neural network) training.
5. Text generation (model evaluation).


In [113]:
import string
import os

### Data loading: Shakespeare

Shakespeare sonnets are awailable at this [link](http://www.gutenberg.org/ebooks/1041?msg=welcome_stranger). In addition, they are stored in the same directory as this notebook (`sonnetes.txt`). Simple preprocessing is already done for you in the next cell: all technical info is dropped.

In [114]:
if not os.path.exists('sonnets.txt'):
    !wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/master/homeworks_basic/Lab2_DL/sonnets.txt

with open('sonnets.txt', 'r') as iofile:
    text = iofile.readlines()
    
TEXT_START = 45
TEXT_END = -368
text = text[TEXT_START : TEXT_END]
assert len(text) == 2616

In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [115]:
# Join all the strings into one and lowercase it
# Put result into variable text.

# Your great code here
s = ''
text = s.join(text).lower()
print(text)
assert len(text) == 100225, 'Are you sure you have concatenated all the strings?'
assert not any([x in set(text) for x in string.ascii_uppercase]), 'Uppercase letters are present'
print('OK!')

  from fairest creatures we desire increase,
  that thereby beauty's rose might never die,
  but as the riper should by time decease,
  his tender heir might bear his memory:
  but thou, contracted to thine own bright eyes,
  feed'st thy light's flame with self-substantial fuel,
  making a famine where abundance lies,
  thy self thy foe, to thy sweet self too cruel:
  thou that art now the world's fresh ornament,
  and only herald to the gaudy spring,
  within thine own bud buriest thy content,
  and tender churl mak'st waste in niggarding:
    pity the world, or else this glutton be,
    to eat the world's due, by the grave and thee.

  ii

  when forty winters shall besiege thy brow,
  and dig deep trenches in thy beauty's field,
  thy youth's proud livery so gazed on now,
  will be a tatter'd weed of small worth held:
  then being asked, where all thy beauty lies,
  where all the treasure of thy lusty days;
  to say, within thine own deep sunken eyes,
  were an all-eating shame, and

### Data loading: "Евгений Онегин"


In [116]:
!wget https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text_pushkin = iofile.readlines()
    
text_pushkin = [x.replace('\t\t', '') for x in text_pushkin]

--2020-12-13 23:17:23--  https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt.2’

onegin.txt.2        100%[===================>] 256.37K  --.-KB/s    in 0.02s   

2020-12-13 23:17:23 (11.7 MB/s) - ‘onegin.txt.2’ saved [262521/262521]



In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [117]:
# Join all the strings into one and lowercase it
# Put result into variable text.

# Your great code here
s = ''
text_pushkin = s.join(text_pushkin).lower()
print(text_pushkin)


i

«мой дядя самых честных правил,
когда не в шутку занемог,
он уважать себя заставил
и лучше выдумать не мог.
его пример другим наука;
но, боже мой, какая скука
с больным сидеть и день и ночь,
не отходя ни шагу прочь!
какое низкое коварство
полуживого забавлять,
ему подушки поправлять,
печально подносить лекарство,
вздыхать и думать про себя:
когда же черт возьмет тебя!»



ii

так думал молодой повеса,
летя в пыли на почтовых,
всевышней волею зевеса
наследник всех своих родных. —
друзья людмилы и руслана!
с героем моего романа
без предисловий, сей же час
позвольте познакомить вас:
онегин, добрый мой приятель,
родился на брегах невы,
где, может быть, родились вы
или блистали, мой читатель;
там некогда гулял и я:
но вреден север для меня


iii

служив отлично-благородно,
долгами жил его отец,
давал три бала ежегодно
и промотался наконец.
судьба евгения хранила:
сперва madame за ним ходила,
потом monsieur ее сменил;
ребенок был резов, но мил.
monsieur l’abbe€, француз убогой,
чтоб не и

Put all the characters, that you've seen in the text, into variable `tokens`.

In [118]:
tokens = sorted(set(text))
print(len(tokens))

tokens_pushkin = sorted(set(text_pushkin))
len(tokens_pushkin)
#tokens

38


83

In [119]:
import numpy as np

In [120]:
def one_hot_encode(n_classes, y):
    y = y.flatten()
    y_one_hot = np.zeros((len(y), n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y.astype(int)] = 1.
    return y_one_hot

Create dictionary `token_to_idx = {<char>: <index>}` and dictionary `idx_to_token = {<index>: <char>}`

In [121]:
idx = np.arange(len(tokens))

idx_to_token = dict(zip(idx, tokens))

token_to_idx = dict(zip(tokens, idx))
encoded_text = np.array([token_to_idx[ch] for ch in text])
len(encoded_text)

idx_pushkin = np.arange(len(tokens_pushkin))

idx_to_token_pushkin = dict(zip(idx_pushkin, tokens_pushkin))

token_to_idx_pushkin = dict(zip(tokens_pushkin, idx_pushkin))
encoded_text_pushkin = np.array([token_to_idx_pushkin[ch] for ch in text_pushkin])
len(encoded_text_pushkin)

141888

*Comment: in this task we have only 38 different tokens, so let's use one-hot encoding.*

### Building the model

Now we want to build and train recurrent neural net which would be able to something similar to Shakespeare's poetry.

Let's use vanilla RNN, similar to the one created during the lesson.

In [122]:
# Your code here
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchsummary
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import time
from random import sample


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [219]:
n_classes = 38
batch_size = 16
seq_length = 50
n_epochs = 10

In [124]:
class CharRNN(nn.Module):    
    def __init__(self, n_hidden=256, n_embedding=128):
        super().__init__()
        self.n_embedding = n_embedding
        self.n_hidden = n_hidden
        
        self.embedding = nn.Embedding(n_classes, self.n_embedding)
        self.rnn = nn.RNN(self.n_embedding, self.n_hidden, batch_first=True)  
        self.activation = nn.ReLU()    
        self.fc = nn.Linear(self.n_hidden, n_classes)

    def forward(self, input_, h_0=None):
        input_emb = self.embedding(input_)
        if h_0 is None:
            h_0 = torch.zeros((1, input_.size(0), self.n_hidden)).to(device)
        output, h_n = self.rnn(input_emb, h_0)
        output = output.contiguous().view(-1, self.n_hidden)
        output = self.activation(output)
        output = self.fc(output)
        return output, h_n

In [125]:
def get_batches(arr, batch_size, seq_length):
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [226]:
def train_model(net, train_data, val_data, criterion, opt):
    val_loss = []
    train_loss = []

    for epoch in range(n_epochs):
        if epoch % 20 == 19:
            for param in opt.param_groups:
                param['lr'] = param['lr'] / 10
        ep_val_loss = []
        ep_train_loss = []
        net.train()
        for x, y in get_batches(train_data, batch_size, seq_length):
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            targets = targets.reshape((batch_size * seq_length,))
            net.zero_grad()
            output, h = net(inputs.to(device))
            # output = output.reshape((batch_size*seq_length, n_classes))

            loss = criterion(output, targets.to(device))
            loss.backward()
            ep_train_loss.append(loss.item())
            opt.step()
        net.eval()
        for x, y in get_batches(val_data, batch_size, seq_length):
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            targets = targets.reshape((batch_size * seq_length,))
            
            output, val_h = net(inputs.to(device))

            # output = output.reshape((batch_size*seq_length, n_classes))
            loss = criterion(output, targets.to(device))                
            ep_val_loss.append(loss.item())

        train_loss.append(np.mean(ep_train_loss))
        val_loss.append(np.mean(ep_val_loss))

        print(f"\t  training loss: {train_loss[-1]:.6f}")
        print(f"\tvalidation loss: {val_loss[-1]:.6f}")
    return train_loss, val_loss

In [221]:
net = CharRNN().to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters())
train_data, val_data = encoded_text[:len(encoded_text)*8//10], encoded_text[len(encoded_text)*8//10:]
train_loss, val_loss = train_model(net, train_data, val_data, criterion, opt)

	  training loss: 2.377427
	validation loss: 2.060785
	  training loss: 1.939774
	validation loss: 1.906250
	  training loss: 1.812986
	validation loss: 1.831455
	  training loss: 1.736371
	validation loss: 1.782298
	  training loss: 1.681255
	validation loss: 1.749586
	  training loss: 1.638313
	validation loss: 1.726988
	  training loss: 1.603266
	validation loss: 1.709236
	  training loss: 1.573928
	validation loss: 1.695650
	  training loss: 1.547841
	validation loss: 1.685436
	  training loss: 1.524777
	validation loss: 1.675677


In [222]:
n_classes = 83
batch_size = 16
seq_length = 50
n_epochs = 10

In [129]:
net_pushkin = CharRNN().to(device)
criterion = nn.CrossEntropyLoss()
opt_pushkin = torch.optim.Adam(net_pushkin.parameters())
train_data_pushkin, val_data_pushkin = encoded_text_pushkin[:len(encoded_text_pushkin)*8//10], encoded_text_pushkin[len(encoded_text_pushkin)*8//10:]
train_loss_pushkin, val_loss_pushkin = train_model(net_pushkin, train_data_pushkin, val_data_pushkin, criterion, opt_pushkin)

	  training loss: 2.819827
	validation loss: 2.483696
	  training loss: 2.377403
	validation loss: 2.330540
	  training loss: 2.259711
	validation loss: 2.254937
	  training loss: 2.183452
	validation loss: 2.204435
	  training loss: 2.125445
	validation loss: 2.168942
	  training loss: 2.078473
	validation loss: 2.142417
	  training loss: 2.038289
	validation loss: 2.120982
	  training loss: 2.004153
	validation loss: 2.104340
	  training loss: 1.973938
	validation loss: 2.092372
	  training loss: 1.947310
	validation loss: 2.082446


Plot the loss function (axis X: number of epochs, axis Y: loss function).

In [130]:
import torch.nn.functional as F

In [164]:
def generate_sample(char_rnn, seed_phrase=' hello', max_length=200, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_idx[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64).to(device)
    # hid_state = char_rnn.initial_state(batch_size=1)
    

    hid_state = None
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        # print(x_sequence[:, -1].shape)
        out, hid_state = char_rnn(x_sequence[:, :, i], hid_state)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        # print(x_sequence.shape, x_sequence)
        out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # if hid_state is None:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1])
        # else:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.cpu().numpy()[0]
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(len(tokens), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        # print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix[None, :, :].to(device)], dim=2)
    return ''.join([tokens[ix] for ix in x_sequence.data.cpu().numpy()[0][0]])

In [171]:
def generate_sample_pushkin(char_rnn, seed_phrase=' привет', max_length=200, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_idx_pushkin[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64).to(device)
    # hid_state = char_rnn.initial_state(batch_size=1)
    

    hid_state = None
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        # print(x_sequence[:, -1].shape)
        out, hid_state = char_rnn(x_sequence[:, :, i], hid_state)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        # print(x_sequence.shape, x_sequence)
        out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # if hid_state is None:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1])
        # else:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.cpu().numpy()[0]
        
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(len(tokens_pushkin), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        # print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix[None, :, :].to(device)], dim=2)
        
    return ''.join([tokens_pushkin[ix] for ix in x_sequence.data.cpu().numpy()[0][0]])

In [172]:
# An example of generated text.
print(generate_sample(net))

 hellones in old be then should expen tho self, could dots mborth
  for wheres fleghery pridemry time segure memory grought peartly prowns of apcestern which ervend
  some i but thy uptermpon but myse


In [173]:
  print(generate_sample_pushkin(net_pushkin))

 привете,
падяiн
виденал слопит снученье.



xxii

дои пленовите
две может же жел евгених ностая буитесь одило ждаль,
все друзья,
ее не мне жале;
нуж риззанье тоторка,
пыстерно охоть та не продленной



### More poetic model

Let's use LSTM instead of vanilla RNN and compare the results.

Plot the loss function of the number of epochs. Does the final loss become better?

In [223]:
# Your beautiful code here
class CharLSTM(nn.Module):    
    def __init__(self, n_hidden=256, n_embedding=256, num_layers=1):
        super().__init__()
        self.n_embedding = n_embedding
        self.n_hidden = n_hidden
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(n_classes, self.n_embedding)
        self.activation0 = nn.ELU()
        self.lstm = nn.LSTM(self.n_embedding, self.n_hidden,
                            batch_first=True, dropout=0.5, num_layers=num_layers)     
        self.activation1 = nn.ELU()
        self.dropout = nn.Dropout(0.5)
        self.activation2 = nn.ELU()
        self.fc = nn.Linear(self.n_hidden, n_classes)

    def forward(self, input_, h_0=None, c_0=None):
        input_emb = self.embedding(input_)
        input_emb = self.activation0(input_emb)
        if h_0 is None and c_0 is None:
            h_0 = torch.zeros((self.num_layers, input_.size(0), self.n_hidden)).to(device)
            c_0 = torch.zeros((self.num_layers, input_.size(0), self.n_hidden)).to(device)
        output, (h_n, c_n) = self.lstm(input_emb, (h_0, c_0))
        output = output.contiguous().view(-1, self.n_hidden)
        output = self.dropout(self.activation1(output))
        output = self.fc(self.activation2(output))
        return output, (h_n, c_0)

In [227]:
n_classes = 38
batch_size = 16
seq_length = 50
n_epochs = 40

In [228]:
net_lstm = CharLSTM(n_hidden=256, n_embedding=256).to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(net_lstm.parameters())
train_data, val_data = encoded_text[:len(encoded_text)*8//10], encoded_text[len(encoded_text)*8//10:]
train_loss, val_loss = train_model(net_lstm, train_data, val_data, criterion, opt)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


	  training loss: 2.488320
	validation loss: 2.105719
	  training loss: 2.054462
	validation loss: 1.939388
	  training loss: 1.924068
	validation loss: 1.857589
	  training loss: 1.851222
	validation loss: 1.805112
	  training loss: 1.800214
	validation loss: 1.769617
	  training loss: 1.760568
	validation loss: 1.743091
	  training loss: 1.729797
	validation loss: 1.720878
	  training loss: 1.700819
	validation loss: 1.700519
	  training loss: 1.675757
	validation loss: 1.686086
	  training loss: 1.654911
	validation loss: 1.671898
	  training loss: 1.635184
	validation loss: 1.661504
	  training loss: 1.619321
	validation loss: 1.650870
	  training loss: 1.603719
	validation loss: 1.644416
	  training loss: 1.585184
	validation loss: 1.638692
	  training loss: 1.571583
	validation loss: 1.633178
	  training loss: 1.559837
	validation loss: 1.627543
	  training loss: 1.545155
	validation loss: 1.625580
	  training loss: 1.532498
	validation loss: 1.619539
	  training loss: 1.520932
	

In [231]:
n_classes = 83
batch_size = 16
seq_length = 50
n_epochs = 40

In [232]:
net_lstm_pushkin = CharLSTM().to(device)
criterion = nn.CrossEntropyLoss()
opt_pushkin = torch.optim.Adam(net_lstm_pushkin.parameters())
train_data_pushkin, val_data_pushkin = encoded_text_pushkin[:len(encoded_text_pushkin)*8//10], encoded_text_pushkin[len(encoded_text_pushkin)*8//10:]
train_loss_pushkin, val_loss_pushkin = train_model(net_lstm_pushkin, train_data_pushkin, val_data_pushkin, criterion, opt_pushkin)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


	  training loss: 2.919397
	validation loss: 2.555246
	  training loss: 2.521086
	validation loss: 2.416387
	  training loss: 2.427361
	validation loss: 2.338095
	  training loss: 2.362162
	validation loss: 2.277945
	  training loss: 2.306601
	validation loss: 2.230882
	  training loss: 2.266629
	validation loss: 2.197677
	  training loss: 2.231039
	validation loss: 2.164838
	  training loss: 2.199348
	validation loss: 2.144790
	  training loss: 2.173292
	validation loss: 2.121932
	  training loss: 2.145686
	validation loss: 2.104937
	  training loss: 2.124898
	validation loss: 2.087447
	  training loss: 2.101602
	validation loss: 2.070864
	  training loss: 2.084277
	validation loss: 2.059007
	  training loss: 2.064789
	validation loss: 2.046686
	  training loss: 2.050311
	validation loss: 2.037741
	  training loss: 2.034259
	validation loss: 2.031681
	  training loss: 2.015165
	validation loss: 2.021922
	  training loss: 2.001494
	validation loss: 2.013730
	  training loss: 1.988827
	

In [236]:
def generate_sample_lstm(char_rnn, seed_phrase=' hello', max_length=200, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_idx[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64).to(device)
    # hid_state = char_rnn.initial_state(batch_size=1)
    

    hid_state = None
    c_0 = None
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        # print(x_sequence[:, -1].shape)
        out, (hid_state, c_0) = char_rnn(x_sequence[:, :, i], hid_state, c_0)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        # print(x_sequence.shape, x_sequence)
        out, (hid_state, c_0) = char_rnn(x_sequence[:, :, -1], hid_state, c_0)
        # if hid_state is None:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1])
        # else:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.cpu().numpy()[0]
        
        # sample next token and push it back into x_sequence
        # print(p_next.shape, len(tokens))
        next_ix = np.random.choice(len(tokens), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64).to(device)
        # print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix[None, :, :]], dim=2)
        
    return ''.join([tokens[ix] for ix in x_sequence.data.cpu().numpy()[0][0]])

In [239]:
def generate_sample_lstm_pushkin(char_rnn, seed_phrase=' привет', max_length=200, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_idx_pushkin[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64).to(device)
    # hid_state = char_rnn.initial_state(batch_size=1)
    

    hid_state = None
    c_0 = None
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        # print(x_sequence[:, -1].shape)
        out, (hid_state, c_0) = char_rnn(x_sequence[:, :, i], hid_state, c_0)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        # print(x_sequence.shape, x_sequence)
        out, (hid_state, c_0) = char_rnn(x_sequence[:, :, -1], hid_state, c_0)
        # if hid_state is None:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1])
        # else:
        #     out, hid_state = char_rnn(x_sequence[:, :, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.cpu().numpy()[0]
        
        # sample next token and push it back into x_sequence
        # print(p_next.shape, len(tokens))
        next_ix = np.random.choice(len(tokens_pushkin), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64).to(device)
        # print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix[None, :, :]], dim=2)
        
    return ''.join([tokens_pushkin[ix] for ix in x_sequence.data.cpu().numpy()[0][0]])

In [237]:
print(generate_sample_lstm(net_lstm))

 hellorand to quond thy i' tongrepraz'd dother, are you an pling if-brimy ar accors are,
  sons youxch timin
  a),
   thisels lubthy omey.
  i as wair bach fles;
  nor'd unece eys thes we sbiss?
  liv


In [240]:
print(generate_sample_lstm_pushkin(net_lstm_pushkin))

 приветы,
и та я теньй,
прах дыхной-трачай нечал едай)
уметом нанног
я постохлипизди пранлал,
в тламой;
отась кай еружда на? слазы
ей сти,
в ед и одущать черой,
и дониты ягих грак их,
онь с усесостеск


Generate text using the trained net with different `temperature` parameter: `[0.1, 0.2, 0.5, 1.0, 2.0]`.

Evaluate the results visually, try to interpret them.

In [ ]:
# Text generation with different temperature values here

### Saving and loading models

Save the model to the disk, then load it and generate text. Examples are available [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html]).

In [ ]:
# Saving and loading code here

### References
1. <a href='http://karpathy.github.io/2015/05/21/rnn-effectiveness/'> Andrew Karpathy blog post about RNN. </a> 
There are several examples of genration: Shakespeare texts, Latex formulas, Linux Sourse Code and children names.
2. <a href='https://github.com/karpathy/char-rnn'> Repo with char-rnn code </a>
3. Cool repo with PyTorch examples: [link](https://github.com/spro/practical-pytorch`)